In [4]:
import pandas as pd

# Sample data creation
data = pd.read_csv('/content/asset_1.csv')
alpha_max = data['alpha'].max()
alpha_max

2.0

In [20]:
class StaticThresholdBacktesterWithPnL:
    def __init__(self, data):
        self.data = data.copy()
        self.positions = 0
        self.initial_capital =  0
        self.capital = self.initial_capital
        self.portfolio_value = []
        self.trades = []
        self.data['position'] = 0

    def run_strategy(self, threshold_buy, threshold_sell):
        self.capital = 0
        self.positions = 0
        self.portfolio_value = []
        self.trades = []
        self.data['position'] = 0

        for i in range(len(self.data)):
            serial_no = self.data['serial_num'].iloc[i]
            alpha = self.data['alpha'].iloc[i]
            price = self.data['price'].iloc[i]
            # condition/strategies.
            if alpha > threshold_buy and self.positions == 1:
                self.hold(serial_no, price, i)
            elif alpha > threshold_buy and self.positions == 0:
                self.buy(serial_no, price, i)
            elif alpha<threshold_buy and self.positions==1 and alpha>threshold_sell:
                self.hold(serial_no, price, i)
            elif alpha < threshold_sell and self.positions == 1 :
                self.sell(serial_no, price, i)
            elif alpha < threshold_sell and self.positions == 0 and alpha>0 :
                self.hold(serial_no, price, i)
            elif alpha < 0 and self.positions >=0:
                self.short(serial_no, price, i)
            elif alpha < 0 and self.positions==-1:
                self.hold(serial_no, price, i)
            elif alpha > 0  and self.positions<0:
                self.buy(serial_no, price, i)

        # # liquidate at end
        price = self.data['price'].iloc[-1]
        serial_num = self.data['serial_num'].iloc[-1] + 1
        if self.positions > 0:
            self.positions -= 1
            self.capital += price
            self.data.at[serial_num, 'position'] = self.positions
            self.portfolio_value.append(self.capital)
            self.trades.append({'serial_no': serial_num, 'action': 'sell', 'price': price, 'positions': self.positions})
        return self.calculate_performance(), self.data, self.trades
# buy
    def buy(self, serial_num, price, index):
        self.positions += 1
        self.capital -= price
        self.trades.append({'serial_no': serial_num, 'action': 'buy', 'price': price, 'positions': self.positions})
        self.data.at[index, 'position'] = self.positions
        self.portfolio_value.append(self.capital)
#sell
    def sell(self, serial_num, price, index):
        self.positions -= 1
        self.capital += price
        self.trades.append({'serial_no': serial_num, 'action': 'sell', 'price': price, 'positions': self.positions})
        self.data.at[index, 'position'] = self.positions
        self.portfolio_value.append(self.capital)
# hold
    def hold(self, serial_num, price, index):
        self.trades.append({'serial_no': serial_num, 'action': 'stay', 'price': price, 'positions': self.positions})
        self.portfolio_value.append(self.capital)
        self.data.at[index, 'position'] = self.positions
# short
    def short(self, serial_num, price, index):
        self.positions -= 1
        self.capital += price
        self.trades.append({'serial_no': serial_num, 'action': 'short', 'price': price, 'positions': self.positions})
        self.data.at[index, 'position'] = -1
        self.portfolio_value.append(self.capital)
# final return
    def calculate_performance(self):
      return self.portfolio_value[-1] - self.initial_capital



In [9]:
import numpy as np

In [12]:
# grid search to get optimal threshold to get the maximised pnl
THRESHOLD_BUY = 0
THRESHOLD_SELL = 0
performance1 = data['price'].iloc[0]
for i in np.linspace(0,2,20):
    for j in np.linspace(0,2,20):
        backtester = StaticThresholdBacktesterWithPnL(data)
        if(i>j):
            performance, updated_data = backtester.run_strategy(i,j)
            if performance>performance1:
                performance1=performance
                THRESHOLD_BUY=i
                THRESHOLD_SELL=j
print(THRESHOLD_BUY,THRESHOLD_SELL)


0.7368421052631579 0.42105263157894735


In [26]:
# return with optimised thresholds to get maximum return
backtester = StaticThresholdBacktesterWithPnL(data)
performance_f, updated_data_f,trades_f = backtester.run_strategy(THRESHOLD_BUY, THRESHOLD_SELL)

print(f"Total Return: {performance_f:.2f}")

# positions at every price
print(updated_data_f)
updated_data_f.to_csv('file2.csv')

# all trades at every price
df = pd.DataFrame(trades_f)
df.to_csv('output.csv', index=False)



Total Return: 29961.34
      serial_num         price     alpha  position
0              0  18105.300781  0.000000         0
1              1  18398.960605 -0.630291        -1
2              2  18204.939538 -0.123420        -1
3              3  18339.357782 -0.240239        -1
4              4  18578.084798 -0.612625        -1
...          ...           ...       ...       ...
5995        5995   7035.576315 -0.318083        -1
5996        5996   7232.249459 -0.199650        -1
5997        5997   7224.645159 -0.076274        -1
5998        5998   7167.188831  1.106138         0
5999        5999   7261.970061  0.000000         0

[6000 rows x 4 columns]
[{'serial_no': 1, 'action': 'short', 'price': 18398.960605167085, 'positions': -1}, {'serial_no': 2, 'action': 'stay', 'price': 18204.939538099745, 'positions': -1}, {'serial_no': 3, 'action': 'stay', 'price': 18339.357782256004, 'positions': -1}, {'serial_no': 4, 'action': 'stay', 'price': 18578.084798438016, 'positions': -1}, {'serial_n